# FlowExplainer: Flow-Disentangled Feature Importance

This tutorial covers the `FlowExplainer`, which uses **normalizing flows** to compute feature importance. By the end, you'll understand:

1. How Flow-DFI differs from OT-based methods
2. The difference between CPI and SCPI methods
3. How to use FlowExplainer with default and custom flow models
4. When to choose FlowExplainer over OTExplainer

## Setup

First, let's import the necessary libraries:

In [1]:
import sys
sys.path.insert(0, '../..')  # Add project root to path

import numpy as np
import warnings
warnings.filterwarnings('ignore')

from fdfi.explainers import FlowExplainer, OTExplainer

# Set random seed for reproducibility
np.random.seed(42)

## Background: Why Flow-DFI?

**OTExplainer** and **EOTExplainer** use optimal transport to create a disentangled representation. However, they rely on parametric assumptions (Gaussian or kernel-based).

**FlowExplainer** uses a **normalizing flow** — a learned neural network that maps between the original feature space X and a disentangled latent space Z. This is:

- **More flexible**: Learns complex non-linear transformations
- **Data-driven**: No parametric assumptions
- **Invertible**: Can map X → Z and Z → X

The tradeoff is that it requires training a neural network, which takes longer.

## Create Test Data

Let's create data with correlated features where only some features are truly important:

In [2]:
# Create correlated features
n_samples = 500
n_features = 8

# Covariance matrix with correlations
cov = np.eye(n_features)
cov[0, 1] = cov[1, 0] = 0.7  # Features 0 and 1 are correlated
cov[2, 3] = cov[3, 2] = 0.5  # Features 2 and 3 are correlated

X = np.random.multivariate_normal(np.zeros(n_features), cov, size=n_samples)

# Model: only features 0, 1, 2 are active
def model(X):
    return X[:, 0] + 2 * X[:, 1] + 0.5 * X[:, 2]

# Split into train/test
X_train, X_test = X[:400], X[400:]

print(f"Training data: {X_train.shape}")
print(f"Test data: {X_test.shape}")
print(f"\nActive features: 0, 1, 2")
print(f"Null features: 3, 4, 5, 6, 7")

Training data: (400, 8)
Test data: (100, 8)

Active features: 0, 1, 2
Null features: 3, 4, 5, 6, 7


## Basic Usage: CPI Method (Default)

The simplest way to use FlowExplainer is with the default CPI (Conditional Permutation Importance) method:

In [3]:
# Create FlowExplainer with CPI method
explainer_cpi = FlowExplainer(
    model,
    data=X_train,
    method='cpi',        # Conditional Permutation Importance
    nsamples=30,         # Monte Carlo samples per feature
    num_steps=200,       # Flow training steps (use more for better results)
    random_state=42,
)

# Compute importance
results_cpi = explainer_cpi(X_test)

print("\nCPI Feature Importance:")
print("-" * 40)
for i, phi in enumerate(results_cpi['phi_X']):
    marker = "*" if i < 3 else ""  # Mark active features
    print(f"  Feature {i}: {phi:8.4f} {marker}")
print("\n* = active feature")

Training complete: 200 steps, final loss=1.7683



CPI Feature Importance:
----------------------------------------
  Feature 0:   7.0929 *
  Feature 1:  10.7967 *
  Feature 2:   0.3819 *
  Feature 3:   0.1437 
  Feature 4:   0.0559 
  Feature 5:   0.0583 
  Feature 6:   0.0103 
  Feature 7:   0.0334 

* = active feature


## CPI vs SCPI: What's the Difference?

FlowExplainer provides two methods for computing importance in the **latent Z-space**:

**CPI (Conditional Permutation Importance)**
- Squared difference after averaging predictions
- Formula: $\phi_{Z,j}^{CPI} = (Y - \mathbb{E}_b[f(\tilde{X}_b^{(j)})])^2$

**SCPI (Sobol-CPI)**
- Conditional variance of counterfactual predictions
- Formula: $\phi_{Z,j}^{SCPI} = \text{Var}_b[f(\tilde{X}_b^{(j)})]$
- Related to Sobol total-order sensitivity indices

For **L2 loss** with independent (disentangled) features, **CPI and SCPI give similar results**, since both measure how much the model output changes when feature $j$ is permuted.

**Jacobian Transformation to X-space**

Both methods compute importance in Z-space (disentangled features). To attribute importance to the **original features** $X_l$, FlowExplainer uses the **Jacobian** $H = \frac{\partial X}{\partial Z}$:

$$\phi_{X,l} = \sum_{k=1}^{d} H_{lk}^2 \cdot \phi_{Z,k}$$

This correctly accounts for how each latent dimension affects each original feature.

In [4]:
# Create FlowExplainer with SCPI method
explainer_scpi = FlowExplainer(
    model,
    data=X_train,
    method='scpi',       # Sobol-CPI
    nsamples=30,
    num_steps=200,
    random_state=42,
)

results_scpi = explainer_scpi(X_test)

# Compare CPI vs SCPI
print("Comparison: CPI vs SCPI")
print("-" * 50)
print(f"{'Feature':>8} {'CPI':>12} {'SCPI':>12} {'Active':>10}")
print("-" * 50)
for i in range(n_features):
    active = "Yes" if i < 3 else "No"
    print(f"{i:>8} {results_cpi['phi_X'][i]:>12.4f} {results_scpi['phi_X'][i]:>12.4f} {active:>10}")

Training complete: 200 steps, final loss=1.7733


Comparison: CPI vs SCPI
--------------------------------------------------
 Feature          CPI         SCPI     Active
--------------------------------------------------
       0       7.0929       6.8508        Yes
       1      10.7967       9.8425        Yes
       2       0.3819       0.3062        Yes
       3       0.1437       0.0840         No
       4       0.0559       0.0236         No
       5       0.0583       0.0111         No
       6       0.0103       0.0050         No
       7       0.0334       0.0438         No


## Computing Both Methods at Once

Use `method='both'` to compute CPI and SCPI simultaneously (more efficient than running twice):

In [5]:
# Compute both CPI and SCPI
explainer_both = FlowExplainer(
    model,
    data=X_train,
    method='both',
    nsamples=30,
    num_steps=200,
    random_state=42,
)

results_both = explainer_both(X_test)

print("Result keys:", list(results_both.keys()))
print("\nCPI importance (phi_Z):", results_both['phi_Z'][:3].round(4))
print("SCPI importance (phi_Z_scpi):", results_both['phi_Z_scpi'][:3].round(4))

Training complete: 200 steps, final loss=1.7355


Result keys: ['phi_Z', 'std_Z', 'se_Z', 'phi_X', 'std_X', 'se_X', 'phi_Z_scpi', 'std_Z_scpi', 'se_Z_scpi', 'phi_X_scpi', 'std_X_scpi', 'se_X_scpi']

CPI importance (phi_Z): [4.7299 7.6382 0.3533]
SCPI importance (phi_Z_scpi): [4.4189 6.87   0.2783]


## Confidence Intervals

FlowExplainer supports the same `conf_int()` method as other explainers:

In [6]:
# Compute confidence intervals
ci = explainer_cpi.conf_int(alpha=0.05, target='X', alternative='greater')

print("\nConfidence Intervals (95%, one-sided):")
print("-" * 70)
print(f"{'Feature':>8} {'Estimate':>10} {'SE':>10} {'CI Lower':>10} {'Significant':>12}")
print("-" * 70)
for i in range(n_features):
    sig = "Yes *" if ci['reject_null'][i] else "No"
    print(f"{i:>8} {ci['phi_hat'][i]:>10.4f} {ci['se'][i]:>10.4f} "
          f"{ci['ci_lower'][i]:>10.4f} {sig:>12}")


Confidence Intervals (95%, one-sided):
----------------------------------------------------------------------
 Feature   Estimate         SE   CI Lower  Significant
----------------------------------------------------------------------
       0     7.0929     0.7590     5.8445        Yes *
       1    10.7967     1.2248     8.7822        Yes *
       2     0.3819     0.0527     0.2951        Yes *
       3     0.1437     0.0173     0.1153        Yes *
       4     0.0559     0.0156     0.0302        Yes *
       5     0.0583     0.0119     0.0386        Yes *
       6     0.0103     0.0101    -0.0062           No
       7     0.0334     0.0106     0.0160        Yes *


## Sampling Methods

FlowExplainer supports different ways to generate counterfactual values in Z-space:

- `'resample'`: Sample from background data (default)
- `'permutation'`: Permute within test set
- `'normal'`: Sample from standard normal
- `'condperm'`: Conditional permutation (regress Z_j | Z_{-j})

In [7]:
# Try different sampling methods
sampling_methods = ['resample', 'permutation', 'normal']
results_by_method = {}

for method in sampling_methods:
    exp = FlowExplainer(
        model, X_train,
        sampling_method=method,
        nsamples=30,
        num_steps=200,
        random_state=42,
    )
    results_by_method[method] = exp(X_test)

print("Importance by Sampling Method:")
print("-" * 55)
print(f"{'Feature':>8} {'resample':>12} {'permutation':>12} {'normal':>12}")
print("-" * 55)
for i in range(n_features):
    r = results_by_method['resample']['phi_X'][i]
    p = results_by_method['permutation']['phi_X'][i]
    n = results_by_method['normal']['phi_X'][i]
    print(f"{i:>8} {r:>12.4f} {p:>12.4f} {n:>12.4f}")

Training complete: 200 steps, final loss=1.6822


Training complete: 200 steps, final loss=1.6810


Training complete: 200 steps, final loss=1.7802


Importance by Sampling Method:
-------------------------------------------------------
 Feature     resample  permutation       normal
-------------------------------------------------------
       0       7.5032       7.1106       6.2776
       1      10.8725      10.2875       9.8244
       2       0.4309       0.6010       0.3521
       3       0.1071       0.1664       0.1567
       4       0.0119       0.0145       0.0160
       5       0.0260       0.0174       0.0110
       6       0.0142       0.0224       0.0157
       7       0.0371       0.0259       0.0307


## Using a Custom Flow Model

You can train a flow model separately and pass it to FlowExplainer. This is useful when:
- You want more control over flow training
- You have a pre-trained flow
- You want to use the same flow for multiple explainers

In [8]:
from fdfi.models import FlowMatchingModel

# Train a custom flow model
custom_flow = FlowMatchingModel(
    X=X_train,
    dim=n_features,
    hidden_dim=64,
    num_blocks=2,
)
custom_flow.fit(num_steps=300, verbose='final')

# Use the pre-trained flow
explainer_custom = FlowExplainer(
    model,
    data=X_train,
    flow_model=custom_flow,  # Pass pre-trained flow
    fit_flow=False,          # Don't retrain
    nsamples=30,
    random_state=42,
)

results_custom = explainer_custom(X_test)
print("\nImportance with custom flow:", results_custom['phi_X'][:4].round(4))

Training complete: 300 steps, final loss=1.5154



Importance with custom flow: [4.0551 6.3796 0.3789 0.1194]


## Comparison: FlowExplainer vs OTExplainer

Let's compare FlowExplainer to OTExplainer on the same data:

In [9]:
# OTExplainer for comparison
explainer_ot = OTExplainer(
    model,
    data=X_train,
    nsamples=30,
    random_state=42,
)
results_ot = explainer_ot(X_test)

print("Comparison: FlowExplainer vs OTExplainer")
print("-" * 55)
print(f"{'Feature':>8} {'Flow (CPI)':>12} {'Flow (SCPI)':>12} {'OT':>12}")
print("-" * 55)
for i in range(n_features):
    f_cpi = results_cpi['phi_X'][i]
    f_scpi = results_scpi['phi_X'][i]
    ot = results_ot['phi_X'][i]
    print(f"{i:>8} {f_cpi:>12.4f} {f_scpi:>12.4f} {ot:>12.4f}")

# Summary statistics
active_mask = np.array([True, True, True, False, False, False, False, False])

print("\n" + "=" * 55)
print("Summary: Active vs Null Feature Importance")
print("=" * 55)
for name, phi in [('Flow CPI', results_cpi['phi_X']), 
                  ('Flow SCPI', results_scpi['phi_X']),
                  ('OT', results_ot['phi_X'])]:
    active_mean = phi[active_mask].mean()
    null_mean = phi[~active_mask].mean()
    ratio = active_mean / null_mean if null_mean > 0 else float('inf')
    print(f"{name:>12}: active={active_mean:.4f}, null={null_mean:.4f}, ratio={ratio:.2f}x")

Comparison: FlowExplainer vs OTExplainer
-------------------------------------------------------
 Feature   Flow (CPI)  Flow (SCPI)           OT
-------------------------------------------------------
       0       7.0929       6.8508       3.0570
       1      10.7967       9.8425       5.3006
       2       0.3819       0.3062       0.3043
       3       0.1437       0.0840       0.0372
       4       0.0559       0.0236       0.0015
       5       0.0583       0.0111       0.0036
       6       0.0103       0.0050       0.0007
       7       0.0334       0.0438       0.0049

Summary: Active vs Null Feature Importance
    Flow CPI: active=6.0905, null=0.0603, ratio=100.97x
   Flow SCPI: active=5.6665, null=0.0335, ratio=169.11x
          OT: active=2.8873, null=0.0096, ratio=301.79x


## Z-space vs X-space Importance

FlowExplainer provides both:
- **phi_Z**: Importance in the disentangled latent space
- **phi_X**: Importance attributed to original features (via Jacobian)

| Space | Meaning |
|-------|---------|
| Z-space (phi_Z) | Importance of independent latent factors |
| X-space (phi_X) | Importance of original correlated features |

For linear transformations (like OTExplainer), these are related by `phi_X = H^T @ H @ phi_Z` where H is the Cholesky factor. For flows, the Jacobian varies with position.

## When to Use FlowExplainer

## Summary

In this tutorial, you learned:

1. **FlowExplainer** uses normalizing flows for flexible, data-driven feature importance
2. **CPI** averages predictions first, **SCPI** averages squared differences (Sobol-style)
3. Use `method='both'` to compute CPI and SCPI simultaneously
4. Different **sampling methods** offer different tradeoffs
5. You can use **custom flow models** for more control
6. FlowExplainer is best for **non-Gaussian data** with complex dependencies

## Next Steps

- Try FlowExplainer on your own data
- Experiment with different `num_steps` values for flow training
- Compare results across different sampling methods
- Read the API documentation for advanced options